In [3]:
import astropy
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt

# The Problem

The Dataiku framework that we were going to use to extract the calibration scan information and time on source was affected in the JAO Cyberattack. I'm trying to find some other way to get the information. I have two options:
* files from Ignacio that he had on his system including scan_table.csv.gz and sb_science_targets_consolidated_bl.csv
* files from Felix with information that he has gleaned from the achive. I believe that this is all MOUS (not EB). I don't know if the per EB information can be gotten out since we process by MOUS

## Playing with Ignacio's data set

In [6]:
scan_table = Table.read('scan_table.csv')

In [7]:
scan_table.keys()

['EXECBLOCK_UID',
 'scanNumber',
 'startTime',
 'endTime',
 'numIntent',
 'numSubscan',
 'scanIntent',
 'calDataType',
 'numField',
 'sourceName']

Hmmmm.... looks like they key off of EXECBLOCK_UID. Not what I have for the other stuff (MOUS). But there might be a relatively straightforward way to map EXECBLOCK_UID to MEMBER_OUS_UID. Felix might have something for this.

In [8]:
len(scan_table)

87268

In [13]:
np.sum(scan_table['scanIntent'] == 'OBSERVE_TARGET')

0

In [14]:
np.unique(scan_table['scanIntent'])

CALIBRATE_ATMOSPHERE
CALIBRATE_BANDPASS
CALIBRATE_DELAY
CALIBRATE_FLUX
CALIBRATE_FOCUS
CALIBRATE_PHASE
CALIBRATE_POINTING
CALIBRATE_WVR
OBSERVE_CHECK_SOURCE


Okay, so it only has the information for the calibration and none on the science sources

In [15]:
scan_table[0:10]

EXECBLOCK_UID,scanNumber,startTime,endTime,numIntent,numSubscan,scanIntent,calDataType,numField,sourceName
str25,int64,str24,str24,int64,int64,str20,str55,int64,str10
uid://A002/Xc5b102/X2ba2,1,2017-10-14T13:46:03.185Z,2017-10-14T13:48:09.017Z,2,10,CALIBRATE_POINTING,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,1,2017-10-14T13:46:03.185Z,2017-10-14T13:48:09.017Z,2,10,CALIBRATE_WVR,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,2,2017-10-14T13:48:10.111Z,2017-10-14T13:49:26.410Z,2,10,CALIBRATE_POINTING,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,2,2017-10-14T13:48:10.111Z,2017-10-14T13:49:26.410Z,2,10,CALIBRATE_WVR,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,3,2017-10-14T13:49:27.478Z,2017-10-14T13:53:50.160Z,2,14,CALIBRATE_FOCUS,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,3,2017-10-14T13:49:27.478Z,2017-10-14T13:53:50.160Z,2,14,CALIBRATE_WVR,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,4,2017-10-14T13:53:51.645Z,2017-10-14T13:55:08.897Z,2,10,CALIBRATE_POINTING,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,4,2017-10-14T13:53:51.645Z,2017-10-14T13:55:08.897Z,2,10,CALIBRATE_WVR,"[""CHANNEL_AVERAGE_CROSS"",""WVR""]",1,J1058+0133
uid://A002/Xc5b102/X2ba2,5,2017-10-14T13:55:10.231Z,2017-10-14T13:56:38.647Z,2,3,CALIBRATE_ATMOSPHERE,"[""FULL_RESOLUTION_AUTO"",""WVR""]",1,J1058+0133


But the startTime and endTime could be used to get some time on the calibration sources for different intents

In [55]:
idx = scan_table['EXECBLOCK_UID'] == 'uid://A002/Xedbc85/X78df'
scan_table[idx]

EXECBLOCK_UID,scanNumber,startTime,endTime,numIntent,numSubscan,scanIntent,calDataType,numField,sourceName
str25,int64,str24,str24,int64,int64,str20,str55,int64,str10


In [42]:
ignacio2 = Table.read('sb_science_targets_consolidated_bl.csv')

In [50]:
ignacio2.columns

<TableColumns names=('OBSPROJECT_UID','SB_UID','SB_NAME','MODE_NAME','ARRAY','REPRESENTATIVE_TARGET_REF','TARGET_ID','SPEC_REF','FIELD_REF','OBSERVING_PARAMETER_REF','SCIPARAM_NAME','SCIPARAM_REPRESENTATIVE_BANDWIDTH','SCIPARAM_REPRESENTATIVE_BANDWIDTH_UNIT','SCIPARAM_REPRESENTATIVE_FREQUENCY','SCIPARAM_REPRESENTATIVE_FREQUENCY_UNIT','SCIPARAM_SENSITIVITY_GOAL','SCIPARAM_SENSITIVITY_GOAL_UNIT','SCIPARAM_INTEGRATION_TIME','SCIPARAM_INTEGRATION_TIME_UNIT','SCIPARAM_SUB_SCAN_DURATION','SCIPARAM_SUB_SCAN_DURATION_UNIT','SCIPARAM_FORCE_ATM_CAL','FIELD_SOLAR_SYSTEM_OBJECT','FIELD_SOURCE_LONG','FIELD_SOURCE_LONG_UNIT','FIELD_SOURCE_LAT','FIELD_SOURCE_LAT_UNIT','FIELD_FIELD_NAME','FIELD_SOURCE_NAME','FIELD_SOURCE_CENTER_VELOCITY','FIELD_SOURCE_CENTER_VELOCITY_UNIT','FIELD_SOURCE_VELOCITY_REF_SYSTEM','FIELD_SOURCE_VELOCITY_DOPPLER_CALC','FIELD_PM_RA','FIELD_PM_RA_UNIT','FIELD_PM_DEC','FIELD_PM_DEC_UNIT','FIELD_NON_SIDEREAL_MOTION','FIELD_PARALLAX','FIELD_PARALLAX_UNIT','FIELD_NAME','BLSPEC_NAME

In [54]:
ignacio2[0:10]['OBSPROJECT_UID','SB_UID','SB_NAME','MODE_NAME','SCIPARAM_INTEGRATION_TIME','SCIPARAM_INTEGRATION_TIME_UNIT','SCIPARAM_SUB_SCAN_DURATION']

OBSPROJECT_UID,SB_UID,SB_NAME,MODE_NAME,SCIPARAM_INTEGRATION_TIME,SCIPARAM_INTEGRATION_TIME_UNIT,SCIPARAM_SUB_SCAN_DURATION
str23,str23,str41,str33,float64,str3,float64
uid://A001/X12d6/X3d2,uid://A001/X1301/X4d2,R_136_d_07_TM1,Standard Interferometry,2872.7,s,30.24
uid://A001/X13b9/X14b0,uid://A001/X1465/X489,12951_a_06_TM1,Standard Interferometry,489.78799999999995,s,18.144
uid://A001/X12eb/X845,uid://A001/X1302/X35f,ad3a-206_bh_03_TM1,Standard Interferometry,24.092,s,24.192
uid://A001/X1525/X1908,uid://A001/X1525/X207e,M12_2573_n_06_TM1,Standard Interferometry,18.043999999999997,s,18.144
uid://A001/X1525/X1908,uid://A001/X1525/X207e,M12_2573_n_06_TM1,Standard Interferometry,18.043999999999997,s,18.144
uid://A001/X1525/Xcd1,uid://A001/X1590/X17b9,ad3a-216_a_03_TM1,Standard Interferometry,36.188,s,36.288
uid://A001/X15b9/X267,uid://A001/X15ba/X539e,HD313571_a_06_TM1,Standard Interferometry,574.4599999999999,s,30.24
uid://A001/X15ba/X241,uid://A001/X2d1f/Xa4a,B335_a_03_TM2,Standard Interferometry,1058.3,s,30.24
uid://A001/X1ee/Xa76,uid://A001/X2fb/X248,NGC_3627_a_06_TE,Standard Interferometry,2086.46,s,18.144


This does have integration time but only for science sources.

## Playing with Felix's data set

Info from Felix's email:

the sum of the integration (in seconds) time per project, MOUS, band,
scan intent and asdm. Of course I can throw in other metadata and/or do
a different grouping etc.

For mosaics, the actual integration time is computed here, not the
average integration time per position in the footprint as we show on the AQ.

In [38]:
felix = Table.read('project_mous_band_asdm_intent_sum-inttime',format='ascii',delimiter='|',
                   guess=False,
                    names=('project_id','mous','band','asdm','intent','sum','na'))

In [39]:
felix.columns

<TableColumns names=('project_id','mous','band','asdm','intent','sum','na')>

In [40]:
idx = felix['mous'] == 'uid://A001/X1465/X103e'
felix[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2019.1.01326.S,uid://A001/X1465/X103e,6,uid://A002/Xedbc85/X78df,BANDPASS FLUX WVR,604.8,--
2019.1.01326.S,uid://A001/X1465/X103e,6,uid://A002/Xedbc85/X78df,PHASE WVR,423.36,--
2019.1.01326.S,uid://A001/X1465/X103e,6,uid://A002/Xedbc85/X78df,TARGET,2802.24,--
2019.1.01326.S,uid://A001/X1465/X103e,6,uid://A002/Xef72bb/Xad5,BANDPASS FLUX WVR,604.8,--
2019.1.01326.S,uid://A001/X1465/X103e,6,uid://A002/Xef72bb/Xad5,PHASE WVR,423.36,--
2019.1.01326.S,uid://A001/X1465/X103e,6,uid://A002/Xef72bb/Xad5,TARGET,2802.24,--


Are the ASDMs the EBs? Each execution block generates an asdm

Can I assume that the nchan etc is the same as the observation setup? Because the WVR is also in here as well. Or does that just play into the data rate?

Do I want to link these tables to my main table somehow? Really I need both.

Now this gets us the time on source assuming no changes to the current observing procedures. Would need to scale some how for WSU. That could get interesting.....

In [56]:
idx = felix['band'] == '10'
felix[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc1e2be/X3900,BANDPASS WVR,604.8,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc1e2be/X3900,CHECK WVR,725.76,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc1e2be/X3900,FLUX WVR,302.4,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc1e2be/X3900,PHASE WVR,1451.52,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc1e2be/X3900,TARGET,1975.68,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc4c2da/X7ba8,BANDPASS WVR,604.8,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc4c2da/X7ba8,CHECK WVR,725.76,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc4c2da/X7ba8,FLUX WVR,302.4,--
2015.1.00217.S,uid://A001/X2f7/X138,10,uid://A002/Xc4c2da/X7ba8,PHASE WVR,1451.52,--


Check sources show up.

## Cross checking information in Felix's data base with the benchmark

The goal here is to make sure that I understand the information and what's in the data base. I should also cross check with the info in Ignacio's scan information csv. He has it broken out a bit differently there.

In [57]:
idx = felix['mous'] == 'uid://A001/X1465/X3a33'
felix[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2019.1.00092.S,uid://A001/X1465/X3a33,4,uid://A002/Xe32bed/X7e4f,BANDPASS FLUX WVR,1512.0,--
2019.1.00092.S,uid://A001/X1465/X3a33,4,uid://A002/Xe32bed/X7e4f,PHASE WVR,302.4,--
2019.1.00092.S,uid://A001/X1465/X3a33,4,uid://A002/Xe32bed/X7e4f,TARGET,2268.0,--


Info from the PL2022 Benchmark:

Time on target source

In [58]:
# Initial time given
38*60

2280

In [62]:
# time given in targets.ms
37*60+57

2277

In [60]:
# time on bandpass from listobs
302.6+303.3+303+302.7+303.4

1515.0

In [61]:
# time on phase cal. each scan was ~30s. I didn't do the detailed math for each scan.
# Note that there were two phase cal for each intermediate phase cal because this was a spectral scan case.
30+60*4+30

300

In [63]:
# Is this in ignacio's data base?
idx = scan_table['EXECBLOCK_UID'] == 'uid://A002/Xe32bed/X7e4f'
scan_table[idx]

EXECBLOCK_UID,scanNumber,startTime,endTime,numIntent,numSubscan,scanIntent,calDataType,numField,sourceName
str25,int64,str24,str24,int64,int64,str20,str55,int64,str10


Not in ignacio's scan_info data base.

In [64]:
idx = felix['mous'] == 'uid://A001/X14d8/X3eb'
felix[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2019.1.00678.S,uid://A001/X14d8/X3eb,3,uid://A002/Xe6a684/X7c41,BANDPASS FLUX WVR,302.4,--
2019.1.00678.S,uid://A001/X14d8/X3eb,3,uid://A002/Xe6a684/X7c41,PHASE WVR,211.68,--
2019.1.00678.S,uid://A001/X14d8/X3eb,3,uid://A002/Xe6a684/X7c41,TARGET,2903.04,--


In [65]:
# time on source:
48*60+39

2919

In [66]:
# time on source in targets.ms
48*60+23

2903

That matches the above rather well.

In [67]:
# BP
5*60+2.9

302.9

In [68]:
# phase
7*30

210

In [69]:
idx = felix['mous'] == 'uid://A001/X1465/X1b3c'
felix[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2019.1.01056.S,uid://A001/X1465/X1b3c,6,uid://A002/Xe1f219/X6d0b,BANDPASS FLUX WVR,604.8,--
2019.1.01056.S,uid://A001/X1465/X1b3c,6,uid://A002/Xe1f219/X6d0b,PHASE WVR,423.36,--
2019.1.01056.S,uid://A001/X1465/X1b3c,6,uid://A002/Xe1f219/X6d0b,TARGET,2701.44,--
2019.1.01056.S,uid://A001/X1465/X1b3c,6,uid://A002/Xe1f219/X7ee8,BANDPASS FLUX WVR,604.8,--
2019.1.01056.S,uid://A001/X1465/X1b3c,6,uid://A002/Xe1f219/X7ee8,PHASE WVR,423.36,--
2019.1.01056.S,uid://A001/X1465/X1b3c,6,uid://A002/Xe1f219/X7ee8,TARGET,2701.44,--


In [72]:
# target
46*60+12

2772

In [73]:
# bp
10*60+5.2

605.2

In [75]:
# phase
60.4*7

422.8

In [77]:
idx = felix['mous'] == 'uid://A001/X133d/X2067'
felix[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xa303,BANDPASS FLUX WVR,302.4,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xa303,CHECK WVR,90.72,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xa303,PHASE WVR,961.632,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xa303,TARGET,2884.896,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xb167,BANDPASS FLUX WVR,302.4,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xb167,CHECK WVR,90.72,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xb167,PHASE WVR,961.632,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xde9c3e/Xb167,TARGET,2884.896,--
2018.1.00566.S,uid://A001/X133d/X2067,6,uid://A002/Xded57c/X2df4,BANDPASS FLUX WVR,302.4,--


In [87]:
# target
48*60+37

2917

In [84]:
#BP
5*60+2.6

302.6

In [86]:
# Phase
18.9*53

1001.6999999999999

In [88]:
# check sour
5*18.9

94.5

Given the results above, I think I understand the table from Felix and am relatively confident in the results.

One question I want to ask Felix is how where the times derived. Consistent with listobs, but not exact.

## Working out how to categorize this information for calculations

I think I'm going to do the calculations on the actual file and then do a table join to add the information to the current data base?

Data base is currently

In [89]:
import large_cubes
from importlib import reload

In [91]:
reload(large_cubes)
test = large_cubes.calc_time_on_source('project_mous_band_asdm_intent_sum-inttime',include_cal=True)

Intent not recognized: BANDPASS DIFFGAIN FLUX PHASE WVR
Intent not recognized: BANDPASS DIFFGAIN FLUX PHASE WVR
Intent not recognized: BANDPASS DIFFGAIN FLUX PHASE WVR
Intent not recognized: DIFFGAIN PHASE WVR
Intent not recognized: DIFFGAIN PHASE WVR
Intent not recognized: DIFFGAIN PHASE WVR
Intent not recognized: DIFFGAIN PHASE WVR
Intent not recognized: DIFFGAIN PHASE WVR
Intent not recognized: BANDPASS DIFFGAIN FLUX PHASE WVR
Intent not recognized: BANDPASS DIFFGAIN FLUX PHASE WVR
Intent not recognized: BANDPASS PHASE WVR
Intent not recognized: BANDPASS PHASE WVR
Intent not recognized: APPPHASE_ACTIVE
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE
Intent not recognized: APPPHA

Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE PHASE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE BANDPASS FLUX POLARIZATION WVR


Intent not recognized: APPPHASE_ACTIVE BANDPASS FLUX POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE POLARIZATION WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACTIVE WVR
Intent not recognized: APPPHASE_ACTIVE TARGET WVR
Intent not recognized: APPPHASE_ACT

In [92]:
test

project_id,mous,bp_time_s,flux_time_s,phase_time_s,pol_time_s,check_time_s,src_time_s
str32,str32,float64,float64,float64,float64,float64,float64
2019.1.01326.S,uid://A001/X1465/X1002,604.8,0.0,181.44,0.0,0.0,997.92
2019.1.01326.S,uid://A001/X1465/X1004,0.0,0.0,0.0,0.0,0.0,2227.68
2019.1.01326.S,uid://A001/X1465/X1008,604.8,0.0,120.96,0.0,0.0,393.12
2019.1.01326.S,uid://A001/X1465/X100a,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X100e,604.8,0.0,120.96,0.0,0.0,393.12
2019.1.01326.S,uid://A001/X1465/X1010,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1014,604.8,0.0,120.96,0.0,0.0,393.12
2019.1.01326.S,uid://A001/X1465/X1016,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X101a,604.8,0.0,120.96,0.0,0.0,393.12


In [96]:
len(test)

6320

In [97]:
np.sum(test['bp_time_s'] >0)

5766

In [100]:
np.sum( (test['bp_time_s'] > 0) & (test['phase_time_s'] > 0))

5749

In [106]:
np.sum(np.invert( test['bp_time_s'] > 0))

554

In [109]:
idx = np.invert(test['bp_time_s'] > 0)
test[idx]

project_id,mous,bp_time_s,flux_time_s,phase_time_s,pol_time_s,check_time_s,src_time_s
str32,str32,float64,float64,float64,float64,float64,float64
2019.1.01326.S,uid://A001/X1465/X1004,0.0,0.0,0.0,0.0,0.0,2227.68
2019.1.01326.S,uid://A001/X1465/X100a,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1010,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1016,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X101c,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1022,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1028,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X102e,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1034,0.0,0.0,0.0,0.0,0.0,2895.168


In [110]:
idx = np.invert(test['bp_time_s'] > 0) & (test['src_time_s'] > 0)
test[idx]


project_id,mous,bp_time_s,flux_time_s,phase_time_s,pol_time_s,check_time_s,src_time_s
str32,str32,float64,float64,float64,float64,float64,float64
2019.1.01326.S,uid://A001/X1465/X1004,0.0,0.0,0.0,0.0,0.0,2227.68
2019.1.01326.S,uid://A001/X1465/X100a,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1010,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1016,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X101c,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1022,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1028,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X102e,0.0,0.0,0.0,0.0,0.0,3015.936
2019.1.01326.S,uid://A001/X1465/X1034,0.0,0.0,0.0,0.0,0.0,2895.168


In [94]:
cal_info_file = 'project_mous_band_asdm_intent_sum-inttime'
t = Table.read(cal_info_file, format='ascii',delimiter='|',
                   guess=False,
                   names=('project_id','mous','band','asdm','intent','sum','na'))

In [95]:
idx = (t['project_id'] == '2019.1.01326.S') 
t[idx]

project_id,mous,band,asdm,intent,sum,na
str16,str23,str5,str26,str46,float64,int64
2019.1.01326.S,uid://A001/X1465/X1002,3,uid://A002/Xe5aacf/Xaba3,BANDPASS FLUX WVR,604.8,--
2019.1.01326.S,uid://A001/X1465/X1002,3,uid://A002/Xe5aacf/Xaba3,PHASE WVR,181.44,--
2019.1.01326.S,uid://A001/X1465/X1002,3,uid://A002/Xe5aacf/Xaba3,TARGET,997.92,--
2019.1.01326.S,uid://A001/X1465/X1002,3,uid://A002/Xe7b231/X9a40,BANDPASS FLUX WVR,604.8,--
2019.1.01326.S,uid://A001/X1465/X1002,3,uid://A002/Xe7b231/X9a40,PHASE WVR,181.44,--
2019.1.01326.S,uid://A001/X1465/X1002,3,uid://A002/Xe7b231/X9a40,TARGET,997.92,--
2019.1.01326.S,uid://A001/X1465/X1004,3,uid://A002/Xe6bd73/X2f2b,TARGET,2227.68,--
2019.1.01326.S,uid://A001/X1465/X1004,3,uid://A002/Xe6e524/Xcf12,TARGET,2227.68,--
2019.1.01326.S,uid://A001/X1465/X1008,3,uid://A002/Xe5aacf/Xa88d,BANDPASS FLUX WVR,604.8,--
